In [ ]:
# -*- coding: utf-8 -*-

In [ ]:
from keras.models import load_model
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

re-size all the images to this

In [ ]:
IMAGE_SIZE = [400, 400]

In [ ]:
train_path = 'ebhi-split-2categorias/train'
valid_path = 'ebhi-split-2categorias/val'
test_path = 'ebhi-split-2categorias/test'

add preprocessing layer to the front of inception

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE +
                        [3], weights='imagenet', include_top=False)

don't train existing weights

In [ ]:
for layer in inception.layers:
    layer.trainable = False

    # useful for getting number of classes
folders = glob(train_path + '/*')

our layers - you can add more if you want

In [ ]:
x = Flatten()(inception.output)
# x = Dense(1000, activation='relu')(x)
#prediction = Dense(len(folders), activation='softmax')(x)
prediction = Dense(1, activation='sigmoid')(x)

create a model object

In [ ]:
model = Model(inputs=inception.input, outputs=prediction)

view the structure of the model

In [ ]:
model.summary()

tell the model what cost and optimization method to use

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(400, 400),
                                                 batch_size=32,
                                                 class_mode='binary')

In [ ]:
valid_set = valid_datagen.flow_from_directory(valid_path,
                                              target_size=(400, 400),
                                              batch_size=32,
                                              class_mode='binary')

In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(400, 400),
                                            batch_size=32,
                                            class_mode='binary')

fit the model

In [ ]:
r = model.fit(
    training_set,
    validation_data=valid_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(valid_set)
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()

In [ ]:
plt.savefig('LossVal_loss_inception')
plt.show()
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()

In [ ]:
plt.savefig('AccVal_acc_inception')
plt.show()

In [ ]:
model.save('hist_model_inception.h5')